# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

1. Таблица books
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.
2. Таблица authors
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.
3. Таблица publishers
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;
4. Таблица ratings
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.
5. Таблица reviews
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

Цели исследования: 

1. Провести анализ данных приобретенного сервиса для чтения книг по подписке
2. Сформулировать возможные рекомендации по продвижению стартапа

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
from IPython.display import Image
from IPython.core.display import HTML
pd.set_option('display.max_rows', None)

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
#Функция для просмотра данных
def show(table):
    return pd.io.sql.read_sql(sql=text(table), con = engine)

In [4]:
#Добавим схему данных для удобства
Image(url= "https://pictures.s3.yandex.net/resources/scheme_1589269096.png")

In [5]:
#Вывод таблицы books
query_books = '''
    SELECT * 
    FROM books 
    LIMIT 5'''
show(query_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
#Вывод таблицы authors
query_books = '''
    SELECT * 
    FROM authors 
    LIMIT 5'''
show(query_books)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
#Вывод таблицы ratings
query_books = '''
    SELECT * 
    FROM ratings 
    LIMIT 5'''
show(query_books)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
#Вывод таблицы reviews
query_books = '''
    SELECT * 
    FROM reviews 
    LIMIT 5'''
show(query_books)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [9]:
#Вывод таблицы publishers
query_books = '''
    SELECT * 
    FROM publishers 
    LIMIT 5'''
show(query_books)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Данные по таблицам вывелись корректно. Также название столбцов таблицы соответствует их содержанию.

## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года;

In [10]:
books_count = '''
    SELECT COUNT(book_id) AS books_cnt 
    FROM books 
    WHERE publication_date > '2000-01-01'
'''
show(books_count)

,books_cnt
0,819


По предоставленным данным количество книг, вышедших после 01.01.2000 составило 819.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [11]:
books_rv_rt = '''
    SELECT b.book_id,
        b.title,
        COUNT(DISTINCT rv.review_id) AS cnt_review,
        AVG(rt.rating) AS avg_rating
    FROM books b LEFT JOIN reviews rv ON b.book_id = rv.book_id
        LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.title
    ORDER BY cnt_review DESC, 
    avg_rating DESC
    LIMIT 15

'''
show(books_rv_rt)

,book_id,title,cnt_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


С увеличением числа обзоров, рейтинг книг становится все более неравномерным. 
- Книга с наибольшим количеством обзоров - Twilight (Twilight #1) (Количество обзоров: 7, Средний рейтинг: 3.662500). 
- Книги с наивысшим рейтингом, такие как:  Arrows of the Queen (Heralds of Valdemar #1), Alas Babylon, Angels Fall и др. имеют рейтинг 5.0, хотя обзоров у них только 2. 
- Имеются книги без обзоров, например, Anne Rice's The Vampire Lestat: A Graphic Novel, The Natural Way to Draw. 
- Книга с самым наименьшим рейтингом - Harvesting the Heart (Средний рейтинг: 1.50).

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [12]:
publish = '''
    SELECT p.publisher,
        COUNT(b.book_id) AS cnt_books
    FROM books b LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY cnt_books DESC
    LIMIT 1
'''
show(publish)

,publisher,cnt_books
0,Penguin Books,42


Издательство выпустившее наибольшее количество книг (не включая брошюры менее 50 стр.) - Penguin Books. 

Количество выпущенных книг: 42.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [13]:
author = '''
    WITH book_rat AS (SELECT a.author,
                        b.title,
                        COUNT(rt.rating_id) AS cnt_ratings,
                        AVG(rt.rating) AS avg_rating
                        
                    FROM authors a LEFT JOIN books b ON a.author_id = b.author_id
                        LEFT JOIN ratings rt ON b.book_id = rt.book_id
                    GROUP BY b.title, a.author
                    HAVING COUNT(rt.rating_id) >= 50
                    ORDER BY cnt_ratings DESC)
    
    SELECT br.author,
        AVG(br.avg_rating) AS avg_rating
    FROM book_rat br
    GROUP BY br.author
    ORDER BY avg_rating DESC
    LIMIT 1
                        
'''
show(author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автором с самой высокой средней оценкой книг является J.K. Rowling/Mary GrandPré. Средняя оценка книг 4.28.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [14]:
review = '''
    WITH review_rat AS (SELECT rt.username,
                            COUNT(rt.rating_id) AS cnt_ratings
                        FROM ratings rt 
                        GROUP BY rt.username
                        HAVING COUNT(rt.rating_id) > 48
                        ORDER BY cnt_ratings DESC)
    SELECT AVG(cnt_review) AS avg_review
    FROM (SELECT RV.username,
                COUNT(rv.review_id) AS cnt_review
            FROM review_rat rr JOIN reviews rv ON rr.username = rv.username
            GROUP BY rv.username) AS avg_data    
    
'''
show(review)

,avg_review
0,24.0


Среднее количество обзоров пользователей, которые поставили более чем по 50 оценок равно 24.

## Дополнительные задания

In [15]:
Image(url= "https://pictures.s3.yandex.net/resources/scheme_1589269096.png")

Выведите таблицу, которая будет содержать по году публикации:

- количество издательств,
- выпущенных книг и
- сколько всего тысяч страниц было в изданных книгах

(отобразить только те года, в которых издано более 30 книг)

In [16]:
dop = '''
    SELECT EXTRACT(YEAR FROM b.publication_date)::int AS year,
            COUNT(DISTINCT p.publisher_id) AS cnt_publishers,
            COUNT(b.book_id) AS cnt_books,
            (SUM(b.num_pages))/1000.0 AS sum_pages_1000
    FROM books b LEFT JOIN publishers p ON b.publisher_id = p.publisher_id
    GROUP BY EXTRACT(YEAR FROM b.publication_date)
    HAVING COUNT(b.book_id) > 30
    ORDER BY year
                        
'''
show(dop)

,year,cnt_publishers,cnt_books,sum_pages_1000
0,1999,26,41,15.763
1,2000,35,38,13.328
2,2001,41,60,21.758
3,2002,62,94,38.597
4,2003,65,105,41.423
5,2004,88,124,46.779
6,2005,89,139,55.967
7,2006,109,184,68.302
8,2007,38,50,18.258


В полученной таблице представлено 9 лет, период которых соствил с 1999 по 2007, в которых издано более 30 книг. Больше всего разных издательств, выпустивших книг в один год было в 2006 - 109 издательств, количество книг выпущенных в этот год составило 184 штуки, суммарное количество страниц в этих книгах превыстило 68 тысяч.

**Вывод:** 

Среди самых востребованных произведений представлены книги, ставшие за последние десятилетия классикой жанров (такие как Сумерки и Гарри Поттер). Популярность книги поддерживается большим количеством обзоров/отзывов на нее. Также стоит отметить, что самыми популярными авторами являются или писатели бестселлеров, или классики.

**Рекомендации для стартапа:** 

- Проанализировать более актуальные данные. 
- Сделать акцент на самых популярных произваедениях. При этом не стоит забывать о создании запоминающегося образа сервиса, возможно следует также провести анализ востребованности новых книг, имеющих высокие рейтинги, но у которых недостаточно отзывов.
- Для получения популярности у пользователей можно организовать акции. Для постоянных пользователей можно сделать бонусы за отзывы о произведениях. В свою очередь отзывы помогут другим пользователям определиться с желанием приобрести книги именно на этом сервисе.

**Действия, которые смогут повысить интерес пользвователей к сервису (доп.анализ для стартапа):**

- Рассмотреть популярность в разрезе одиночных книг и серий.
- Определить, какие жанры более востребованы, и на основе этого обратить внимание на издательства, с которыми можно организовать дальнейшее взаимодействие.